In [1]:
!pip install pillow tqdm
!pip install exif

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 4.0 MB/s eta 0:00:00


## Cell 1: Imports and Paths

In [2]:
import os
from tqdm import tqdm

# Prompt user for source and destination folders
SOURCE_FOLDER = input("Enter the path to the source folder: ").strip()
DEST_FOLDER = input("Enter the path to the destination folder: ").strip()

# Set image size constraints
MAX_WIDTH = 1024
MAX_HEIGHT = 768

# Create destination folder if it doesn't exist
os.makedirs(DEST_FOLDER, exist_ok=True)

Enter the path to the source folder: D:\Downloads\temp_pics
Enter the path to the destination folder: D:\Downloads\temp_pics\webp_resized


## Cell 2: Resize Function with Copyright Overlay

In [3]:
from PIL import Image, ImageDraw, ImageFont
from exif import Image as ExifImage
import json

def resize_image(image_path, output_path, copyright_text="© Bruce Graham Photography | UntamedFrames.ca"):
    with Image.open(image_path) as img:
        img = img.convert("RGB")
        width, height = img.size

        # Calculate scale based on both width and height constraints
        width_scale = MAX_WIDTH / width
        height_scale = MAX_HEIGHT / height
        scale = min(width_scale, height_scale, 1.0)

        new_size = (int(width * scale), int(height * scale))
        resized = img.resize(new_size, Image.LANCZOS)

        # Draw copyright text
        draw = ImageDraw.Draw(resized)
        font_size = int(new_size[1] * 0.025)  # ~2.5% of image height

        try:
            font = ImageFont.truetype("arial.ttf", font_size)
        except:
            font = ImageFont.load_default()

        # Measure text size
        bbox = draw.textbbox((0, 0), copyright_text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]

        # Bottom-right position
        x_br = new_size[0] - text_width - 10
        y_br = new_size[1] - text_height - 10

        # Top-left position
        x_tl = 10
        y_tl = 10

        # Bottom-right: shadow + main text
        draw.text((x_br + 1, y_br + 1), copyright_text, font=font, fill=(0, 0, 0, 100))
        draw.text((x_br, y_br), copyright_text, font=font, fill=(255, 255, 255, 180))

        # Top-left: lighter watermark
        draw.text((x_tl, y_tl), copyright_text, font=font, fill=(255, 255, 255, 100))

        # Save readable EXIF as .json (safe fallback for corrupt headers)
        try:
            with open(image_path, "rb") as f:
                exif_img = ExifImage(f)
                if exif_img.has_exif:
                    exif_dict = {}
                    for tag in exif_img.list_all():
                        try:
                            value = getattr(exif_img, tag)
                            exif_dict[tag] = str(value)
                        except (AttributeError, NotImplementedError, TypeError):
                            continue
                    sidecar_path = output_path.replace(".webp", ".json")
                    with open(sidecar_path, "w", encoding="utf-8") as f_out:
                        json.dump(exif_dict, f_out, indent=2)
        except Exception as e:
            print(f"EXIF skipped for {os.path.basename(image_path)}: {e}")

        # Save as WebP
        resized.save(output_path, "WEBP", quality=85)

## Cell 3: Run the Batch

In [4]:
for filename in tqdm(os.listdir(SOURCE_FOLDER)):
    if filename.lower().endswith((".jpg", ".jpeg", ".png", ".tiff", ".bmp")):
        input_path = os.path.join(SOURCE_FOLDER, filename)
        output_name = os.path.splitext(filename)[0] + ".webp"
        output_path = os.path.join(DEST_FOLDER, output_name)
        resize_image(input_path, output_path)
        print(f"Converted: {filename} → {output_name}")

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Downloads\\temp_pics'